# Aim of this Notebook

**Welcome everyone!**

**In this notebook, I will be dealing with the Digit Recognition by using CNN.**

**For the first step of this notebook, I will do some visualizations.**

**For the next step, I will train a CNN models for the accomplish my aim.**

**I am open to feedback and suggestions, feel free to comment your feedback and suggestions on the comment section or contact me.**

**So, let's get started!**

# Importing Libraries

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

keras = tf.keras
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# Reading Datasets

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
train.head()

In [ ]:
print('Shape of the train data:', train.shape)
print('Shape of the test data:', test.shape)

In [ ]:
train_unique = pd.DataFrame(train['label'].unique()).sort_values(by=0)
train_unique

**Train dataset includes all the labels from the 0 to 9.**

# Simple Visualization

In [ ]:
plt.figure(figsize=(15, 8))
splot = sns.countplot(data=train, x='label',
                      order=train['label'].value_counts().index,
                      edgecolor=(0, 0, 0),
                      linewidth=2)

for p in splot.patches:
    splot.annotate(format(p.get_height(), '.1f'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha='center', va='center',
                   xytext=(0, 9),
                   textcoords='offset points')
plt.ylabel('Frequency of the Labels', fontsize=14)
plt.xlabel('Labels', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Distribution of the Labels of the Train Dataset', fontsize=20)

**According to the figure above, the x-axis represents that the labels of the training dataset are ordered from the high frequent one to lesser. The y-axis represents that the frequency of the labels.**

**As seen from the figure, the number 1 is the most frequent label in the training dataset.**

**According to the graph above, the dataset labels are pretty equally distributed. There are not major frequency differences between the labels.**

# Feature and Label

In [ ]:
y = train['label']  # Labels
X = train.drop(['label'], axis=1)  # Features
X.head()

In [ ]:
# Let's look at the some exapmles of the feature set
for i in range(9):
    plt.subplot(330 + 1 + i)
    fig = X.iloc[i].values.reshape((28, 28))
    plt.imshow(fig)
    plt.axis('off')
plt.show()

# Normalizing and Reshaping

In [ ]:
# Normalizing the dataset
X = X / 255.0

# CNN expects 3D inputs, so I will convert my data tto 3D form
y = y.values.reshape(-1, 1)
X = X.values.reshape(-1, 28, 28, 1)

print('Shape of the X matrix:', X.shape)
print('Shape of the y matrix:', y.shape)

**At the end of this step, we have 3D inputs.**

In [ ]:
# Encoding the labels
y = to_categorical(y, num_classes=10)

# Train-Test Split

In [ ]:
# Train-Test split
trainX, valX, trainY, valY = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=13)

# CNN Model Part-1

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(trainX.shape[1:])))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(10, activation='softmax'))

# CNN Model Part-2

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Adam Optimizer

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', patience=2,
                                                 restore_best_weights=True)

hist = model.fit(trainX, trainY, epochs=10, batch_size=64, callbacks=[early_stopping],
                 verbose=1, validation_data=(valX, valY))
hist.history.keys()
model.summary()

In [ ]:
# Evaluation of the model

model.evaluate(valX, valY)

# Performance Graphs

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Losses of the Model', fontsize=20)
plt.legend(['Train Loss', 'Validation Loss'], loc='upper right')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Losses', fontsize=12)

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy of the Model', fontsize=20)
plt.legend(['Train Accuracy', 'Validation Accuracy'], loc='upper right')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Losses', fontsize=12)

# Test Data

In [ ]:
test = pd.read_csv('../input/digit-recognizer/test.csv')
test = test / 255.0
test = test.values.reshape(-1, 28, 28, 1)

In [ ]:
predictions = model.predict(test)
predictions = np.argmax(predictions, axis=1)

predictions = pd.DataFrame(predictions)
predictions

In [ ]:
sample_sub = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
sample_sub = pd.concat([sample_sub['ImageId'], predictions], axis=1)
sample_sub.columns = ['ImageId', 'Label']
sample_sub

In [ ]:
sample_sub.to_csv('ata_submission.csv', index=False)